In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE

# Load dataset
df = pd.read_csv("creditcard.csv")  # Replace with your dataset path
print("Dataset loaded successfully.")

# Display basic info
print(df.head())
print("\nClass Distribution:\n", df['Class'].value_counts())

# Check for missing values
print("\nMissing values:\n", df.isnull().sum())

# Normalize the 'Amount' column
scaler = StandardScaler()
df['Amount_Norm'] = scaler.fit_transform(df[['Amount']])
df.drop(['Time', 'Amount'], axis=1, inplace=True)

# Define features and target
X = df.drop('Class', axis=1)
y = df['Class']

# Handle imbalance using SMOTE (Synthetic Minority Over-sampling Technique)
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)

print("\nAfter SMOTE:")
print("Resampled Class Distribution:\n", pd.Series(y_res).value_counts())

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_res, y_res, test_size=0.2, random_state=42, stratify=y_res
)

# Train a Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluation
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Accuracy Score:", accuracy_score(y_test, y_pred))

# Feature Importance (Optional)
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]
features = X.columns

# Plot top 10 features
plt.figure(figsize=(10, 6))
plt.title("Feature Importances (Top 10)")
sns.barplot(x=importances[indices[:10]], y=features[indices[:10]])
plt.show()
